# Fashion Product Classifier

## Downloading the Data

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od

In [ ]:
od.download("https://www.kaggle.com/paramaggarwal/fashion-product-images-small")

In [ ]:
cd fashion-product-images-small/

/content/fashion-product-images-small


## Image name with Extension

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("styles.csv",error_bad_lines=False)

b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11\nSkipping line 33020: expected 10 fields, saw 11\nSkipping line 35748: expected 10 fields, saw 11\nSkipping line 35962: expected 10 fields, saw 11\nSkipping line 37770: expected 10 fields, saw 11\nSkipping line 38105: expected 10 fields, saw 11\nSkipping line 38275: ex

In [ ]:
df.head(10)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011.0,Casual,Inkfruit Mens Chain Reaction T-shirt
6,30805,Men,Apparel,Topwear,Shirts,Green,Summer,2012.0,Ethnic,Fabindia Men Striped Green Shirt
7,26960,Women,Apparel,Topwear,Shirts,Purple,Summer,2012.0,Casual,Jealous 21 Women Purple Shirt
8,29114,Men,Accessories,Socks,Socks,Navy Blue,Summer,2012.0,Casual,Puma Men Pack of 3 Socks
9,30039,Men,Accessories,Watches,Watches,Black,Winter,2016.0,Casual,Skagen Men Black Watch


In [ ]:
df['image'] = df.apply(lambda row: str(row['id']) + ".jpg", axis=1)

In [ ]:
df.head(10)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,39386.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,59263.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,53759.jpg
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011.0,Casual,Inkfruit Mens Chain Reaction T-shirt,1855.jpg
6,30805,Men,Apparel,Topwear,Shirts,Green,Summer,2012.0,Ethnic,Fabindia Men Striped Green Shirt,30805.jpg
7,26960,Women,Apparel,Topwear,Shirts,Purple,Summer,2012.0,Casual,Jealous 21 Women Purple Shirt,26960.jpg
8,29114,Men,Accessories,Socks,Socks,Navy Blue,Summer,2012.0,Casual,Puma Men Pack of 3 Socks,29114.jpg
9,30039,Men,Accessories,Watches,Watches,Black,Winter,2016.0,Casual,Skagen Men Black Watch,30039.jpg


In [ ]:
df = df.sample(frac=1)

In [ ]:
df.head(10)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
24718,9236,Men,Apparel,Topwear,Shirts,Brown,Fall,2011.0,Formal,Mark Taylor Men Brown Striped Shirt,9236.jpg
7190,21804,Men,Accessories,Belts,Belts,Brown,Fall,2011.0,Casual,Fossil Men Spencer Brown Belt,21804.jpg
802,45269,Men,Accessories,Watches,Watches,Black,Summer,2017.0,Casual,CASIO Youth Series Digital Men Black Small Dia...,45269.jpg
18896,2251,Men,Footwear,Flip Flops,Flip Flops,Brown,Fall,2010.0,Casual,Puma Mens Winglet Brown Flip Flop,2251.jpg
6837,5141,Women,Apparel,Topwear,Tshirts,Orange,Summer,2011.0,Casual,Wrangler Women Orange T-shirt,5141.jpg
24650,3992,Men,Apparel,Topwear,Tshirts,Black,Summer,2011.0,Casual,Mr.Men Men's Funny Black T-shirt,3992.jpg
32594,18923,Women,Apparel,Topwear,Shirts,Red,Fall,2011.0,Casual,Levis Women Work Western Red Shirt,18923.jpg
11257,16460,Men,Apparel,Topwear,Shirts,Blue,Fall,2011.0,Casual,Locomotive Men Check Blue Shirt,16460.jpg
24122,36796,Men,Accessories,Watches,Watches,Black,Winter,2016.0,Casual,Maxima Men Black Dial Ssteele Watch,36796.jpg
17718,46208,Women,Accessories,Jewellery,Bangle,Black,Winter,2015.0,Casual,Adrika Women Black Bangle,46208.jpg


In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df.head(10)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,7106,Unisex,Accessories,Bags,Backpacks,Black,Winter,2015.0,Casual,Wildcraft Unisex Black Backpack,7106.jpg
1,49738,Men,Accessories,Ties,Ties,Maroon,Summer,2012.0,Casual,Park Avenue Men Maroon Tie,49738.jpg
2,53715,Women,Accessories,Bags,Handbags,Cream,Summer,2012.0,Casual,Kiara Women Cream Handbag,53715.jpg
3,33868,Men,Accessories,Watches,Watches,Black,Winter,2016.0,Casual,Citizen Men Black Dial Chronograph Watch,33868.jpg
4,15898,Unisex,Footwear,Shoes,Casual Shoes,Blue,Winter,2013.0,Casual,Warner Bros Kids Blue Dance Casual Shoes,15898.jpg
5,13676,Men,Footwear,Shoes,Sports Shoes,Silver,Fall,2011.0,Sports,ADIDAS Men Titan Sports Shoes,13676.jpg
6,47165,Men,Accessories,Belts,Belts,Black,Summer,2012.0,Casual,Peter England Men Black Belt,47165.jpg
7,25026,Women,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Sports,Nike Women Miler Blue Jersey,25026.jpg
8,15022,Men,Apparel,Bottomwear,Track Pants,Grey,Fall,2011.0,Sports,ADIDAS Men Solid Grey Track Pants,15022.jpg
9,57225,Women,Accessories,Bags,Handbags,Red,Summer,2012.0,Casual,Murcia Women Red Handbag,57225.jpg


## Preparing the ImageDataGenerator for training the model

In [ ]:
from keras_preprocessing.image import ImageDataGenerator
batch_size = 256

image_generator = ImageDataGenerator(
    validation_split=0.2
)

training_generator = image_generator.flow_from_dataframe(
    dataframe=df,
    directory="images",
    x_col="image",
    y_col="masterCategory",
    target_size=(60,80),
    batch_size=batch_size,
    subset="training"
)

validation_generator = image_generator.flow_from_dataframe(
    dataframe=df,
    directory="images",
    x_col="image",
    y_col="masterCategory",
    target_size=(60,80),
    batch_size=batch_size,
    subset="validation"
)
classes = len(training_generator.class_indices)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 5 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 35536 validated image filenames belonging to 7 classes.
Found 8883 validated image filenames belonging to 7 classes.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 5 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [ ]:
classes

7

## Initializing the Model using Neural Network

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [ ]:
#Initializing our model
classifier = Sequential()

classifier.add(Conv2D(32,(3,3),input_shape = (60,80,3), activation = 'relu'))

classifier.add(MaxPooling2D(pool_size=(3, 3)))

classifier.add(Conv2D(32,(3,3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(3, 3)))

classifier.add(Flatten())

classifier.add(Dense(units=32,activation = 'relu'))
classifier.add(Dense(units=64,activation = 'relu'))
classifier.add(Dense(units=128,activation = 'relu'))
classifier.add(Dense(units=256,activation = 'relu'))
classifier.add(Dense(units=512,activation = 'relu'))

classifier.add(Dense(units=7,activation = 'softmax'))

classifier.compile(optimizer='adam',
              loss="categorical_crossentropy",
              metrics=['accuracy'])

## Fitting Data in our Neural Network

In [ ]:
from math import ceil

classifier.fit_generator(
    generator=training_generator,
    steps_per_epoch=ceil(0.8 * (df.shape[0] / batch_size)),

    validation_data=validation_generator,
    validation_steps=ceil(0.2 * (df.shape[0] / batch_size)),

    epochs=5,
    verbose=1
)

loss, acc = classifier.evaluate_generator(validation_generator, steps=ceil(0.2 * (df.size / batch_size)))
print("\n%s: %.2f%%" % (classifier.metrics_names[1], acc * 100))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/5
139/139 [==============================] - 33s 174ms/step - loss: 0.8150 - accuracy: 0.8048 - val_loss: 0.2392 - val_accuracy: 0.9251
Epoch 2/5
139/139 [==============================] - 23s 168ms/step - loss: 0.1881 - accuracy: 0.9419 - val_loss: 0.1903 - val_accuracy: 0.9415
Epoch 3/5
139/139 [==============================] - 24s 170ms/step - loss: 0.1501 - accuracy: 0.9543 - val_loss: 0.1708 - val_accuracy: 0.9460
Epoch 4/5
139/139 [==============================] - 23s 168ms/step - loss: 0.1274 - accuracy: 0.9616 - val_loss: 0.1555 - val_accuracy: 0.9559
Epoch 5/5
139/139 [==============================] - 23s 168ms/step - loss: 0.1163 - accuracy: 0.9648 - val_loss: 0.1344 - val_accuracy: 0.9577


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  



accuracy: 95.77%


## Saving the Model for further use

In [ ]:
classifier.save("model.h5")

## Testing on our data

In [ ]:
import numpy as np
from keras.preprocessing import image

filename = "15970.jpg"

from keras.models import load_model
new_model = load_model('model.h5')
new_model.summary()

test_image = image.load_img("images/"+filename,target_size=(60,80))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

result = new_model.predict(test_image)
print(result)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 58, 78, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 19, 26, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 17, 24, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 8, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 32)                4

In [ ]:
val = np.argmax(result)
print(val)

1


In [ ]:
my_dict = training_generator.class_indices
key_list = list(my_dict.keys()) 
val_list = list(my_dict.values()) 
print(key_list[val])


Apparel


## Saving the Class IDs

In [ ]:
import pickle
# save the model to disk
filename1 = 'key_list'
filename = 'val_list'
pickle.dump(key_list, open(filename1, 'wb'))
pickle.dump(val_list, open(filename, 'wb'))